In [1]:
#Import Modules
import pyart
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
from io import StringIO
from boto.s3.connection import S3Connection
import shutil, os, sys
import gzip
import pylab as plb
import matplotlib as mpl
import numpy.ma as ma
from pylab import *
print(pyart.__version__)

1.6.0


/home/zsherman/anaconda3/lib/python3.5/site-packages/pyart/map/grid_mapper.py:40: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  from .ball_tree import BallTree
/home/zsherman/anaconda3/lib/python3.5/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/home/zsherman/anaconda3/lib/python3.5/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/home/zsherman/anaconda3/lib/python3.5/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1=

In [3]:
conn = S3Connection(anon=True)
bucket = conn.get_bucket('testradar')
for key in bucket.list():
    print (key.name.encode('utf-8'))
#CPOL (C-band Polarimetric Radar)

b'CSU_unfolded/'
b'CSU_unfolded/CPOL_corr_20060122_140009_vt_unfolded_hand.cfrad.nc.gz'
b'CSU_unfolded/CPOL_corr_20060122_141010_vt_unfolded_hand.cfrad.nc.gz'
b'CSU_unfolded/CPOL_corr_20060122_142009_vt_unfolded_hand.cfrad.nc.gz'
b'CSU_unfolded/CPOL_corr_20060122_143009_vt_unfolded_hand.cfrad.nc.gz'
b'CSU_unfolded/CPOL_corr_20060122_144009_vt_unfolded_hand.cfrad.nc.gz'
b'CSU_unfolded/CPOL_corr_20060122_145009_vt_unfolded_hand.cfrad.nc.gz'
b'zindex.png'


In [4]:
#gzip and opening CPOL files
LOCAL_PATH = '/home/zsherman/Documents/Jupyter/'
filenames = []
bucket_list = bucket.list()
for l in bucket_list:
    keyString = str(l.key)
    print(keyString)
    if '.gz' in keyString:
        l.get_contents_to_filename(LOCAL_PATH+keyString)
        filenames.append(LOCAL_PATH+keyString[0:-3])
        gzfh = gzip.open(LOCAL_PATH+keyString, 'rb')
        ofh = open(LOCAL_PATH+keyString[0:-3], 'wb')
        for line in gzfh:
            ofh.write(line)
        ofh.close()
        gzfh.close()
        os.remove(LOCAL_PATH+keyString)

CSU_unfolded/
CSU_unfolded/CPOL_corr_20060122_140009_vt_unfolded_hand.cfrad.nc.gz
CSU_unfolded/CPOL_corr_20060122_141010_vt_unfolded_hand.cfrad.nc.gz
CSU_unfolded/CPOL_corr_20060122_142009_vt_unfolded_hand.cfrad.nc.gz
CSU_unfolded/CPOL_corr_20060122_143009_vt_unfolded_hand.cfrad.nc.gz
CSU_unfolded/CPOL_corr_20060122_144009_vt_unfolded_hand.cfrad.nc.gz
CSU_unfolded/CPOL_corr_20060122_145009_vt_unfolded_hand.cfrad.nc.gz
zindex.png


In [5]:
#Reading all files with pyart
#Reading all files with pyart
radar_files = []
radar_files += [pyart.io.read(filenames[0]),
               pyart.io.read(filenames[1]),
               pyart.io.read(filenames[2]),
               pyart.io.read(filenames[3]),
               pyart.io.read(filenames[4]),
               pyart.io.read(filenames[5])]

In [6]:
def corrected_velocity_func(radar):
    corrV_reg = pyart.correct.dealias_region_based(radar, interval_splits=5, rays_wrap_around=True, keep_original=False)
    radar.add_field('corrected_velocity_reg', corrV_reg, replace_existing = True)

In [7]:
for radar_file in radar_files:
    corrected_velocity_func(radar_file)

In [8]:
Ny_vel = []
Ny_vel += [radar_files[0].instrument_parameters['nyquist_velocity']['data'][0],
           radar_files[1].instrument_parameters['nyquist_velocity']['data'][0],
           radar_files[2].instrument_parameters['nyquist_velocity']['data'][0],
           radar_files[3].instrument_parameters['nyquist_velocity']['data'][0],
           radar_files[4].instrument_parameters['nyquist_velocity']['data'][0],
           radar_files[5].instrument_parameters['nyquist_velocity']['data'][0]]
Ny_vel

/home/zsherman/anaconda3/lib/python3.5/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/zsherman/anaconda3/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


[13.35, 13.35, 13.35, 13.35, 13.35, 13.35]

In [9]:
velocity = []
velocity += [np.array(radar_files[0].fields['velocity']['data'][0]),
              np.array(radar_files[1].fields['velocity']['data'][0]),
              np.array(radar_files[2].fields['velocity']['data'][0]),
              np.array(radar_files[3].fields['velocity']['data'][0]),
              np.array(radar_files[4].fields['velocity']['data'][0]),
              np.array(radar_files[5].fields['velocity']['data'][0])]
velocity

[array([ -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
         -3.27680000e+04,  -3.27680000e+04,  -3.27680000e+04,
        

In [15]:
radar_files[2].fields['corrected_velocity_reg']['data']

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- 0.0 ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [-- 0.0 0.0 ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True False ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [ True False False ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True False False ...,  True  True  True]],
       fill_value = 1e+20)

In [10]:
def dealiasing_differences_func(radar):
    diff_reg = radar.fields['corrected_velocity_reg']['data'] - radar.fields['velocity']['data']
    diff_reg_sc = diff_reg/Ny_vel[0]
    diff_f = radar.fields['corrected_velocity_reg']
    diff_f['data'] = diff_reg_sc
    diff_f['long_name'] = 'Added Nyquist intervals'
    diff_f['standard_name'] = "added_Ny_intervals"
    diff_f['units']=''
    radar.add_field('dealiasing_differences_sc', diff_f, replace_existing = True)

In [11]:
for radar_file in radar_files:
    dealiasing_differences_func(radar_file)

In [16]:
deal_datas = []
deal_datas += [np.anyarray(radar_files[0].fields['dealiasing_differences_sc']['data'][0]),
              np.array(radar_files[1].fields['dealiasing_differences_sc']['data'][0]),
              np.array(radar_files[2].fields['dealiasing_differences_sc']['data'][0]),
              np.array(radar_files[3].fields['dealiasing_differences_sc']['data'][0]),
              np.array(radar_files[4].fields['dealiasing_differences_sc']['data'][0]),
              np.array(radar_files[5].fields['dealiasing_differences_sc']['data'][0])]
deal_datas

AttributeError: module 'numpy' has no attribute 'anyarray'

In [18]:
np.asarray?